In [24]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [25]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

EXTRACT - CSV

In [82]:
def extract_from_csv(file_to_process):
    dataframe=pd.read_csv(file_to_process)
    return dataframe

EXTRACT - JSON

In [83]:
def extract_from_json(file_to_process):
    dataframe=pd.read_json(file_to_process,lines=True)
    return dataframe

EXTRACT - XML

In [84]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['name','height','weight'])
    tree=ET.parse(file_to_process)
    root=tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        new_row = pd.DataFrame({"name": [name], "height": [height], "weight": [weight]})
        dataframe = pd.concat([dataframe, new_row], ignore_index=True)

        
    return dataframe


Sample testing

In [85]:
extract_from_csv('source1.csv')



,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [86]:
extract_from_json('source1.json')


,name,height,weight
0,jack,68.70,123.30
1,tom,69.80,141.49
2,tracy,70.01,136.46
3,john,67.90,112.37


In [87]:
extract_from_xml('source1.xml')

,name,height,weight
0,simon,67.90,112.37
1,jacob,66.78,120.67
2,cindy,66.49,127.45
3,ivan,67.62,114.14


EXTRACT Functon

In [88]:
#This function is gather all the data from various file types(csv,json,xml) in to a DataFrame

def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        csv_data=extract_from_csv(csvfile)
        extracted_data = pd.concat([extracted_data,csv_data], ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        json_data=extract_from_json(jsonfile)
        extracted_data = pd.concat([extracted_data,json_data], ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        xml_data=extract_from_xml(xmlfile)
        extracted_data = pd.concat([extracted_data,xml_data], ignore_index=True)
        
    return extracted_data

In [89]:
extract().head()

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


TRANSFORM Function

In [90]:
#This function is transform data to another data values.(converts height,weight)

def transform(data):     
      
        data['height'] = round(data.height * 0.0254,2)                #Convert height which is in inches to millimeter 
        data['weight'] = round(data.weight * 0.45359237,2)              #Convert weight which is in pounds to kilograms
        return data







LOADING Function

In [91]:
def load(targetfile,data_to_load):      #This function is load the data into a one csv file
    data_to_load.to_csv(targetfile)  

LOGGING Function

In [92]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

Running ETL Process

In [93]:
log("ETL Job Started")

In [94]:
log('Extract phase started')
extracted_data=extract()
log('Extracted phase ended')
extracted_data.head()

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [95]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data.head()

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45


In [96]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [97]:
log("ETL Job Ended")